 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Functions,-generators-and-data" data-toc-modified-id="Functions,-generators-and-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Functions, generators and data</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Training" data-toc-modified-id="Training-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training</a></span></li></ul></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Only-the-top-part" data-toc-modified-id="Only-the-top-part-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Only the top part</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Only-the-bottom-part" data-toc-modified-id="Only-the-bottom-part-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Only the bottom part</a></span><ul class="toc-item"><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Pseudo-labeling" data-toc-modified-id="Pseudo-labeling-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Pseudo labeling</a></span></li></ul></ul></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Is-the-specialised-model-much-better-than-the-common-model" data-toc-modified-id="Is-the-specialised-model-much-better-than-the-common-model-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Is the specialised model much better than the common model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#top-part" data-toc-modified-id="top-part-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>top part</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#bottom-part" data-toc-modified-id="bottom-part-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>bottom part</a></span></li></ul></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Test-set-predictions" data-toc-modified-id="Test-set-predictions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Test set predictions</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#top-part" data-toc-modified-id="top-part-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>top part</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#Bottom-part" data-toc-modified-id="Bottom-part-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Bottom part</a></span></li><li><span><a href="http://localhost:8889/notebooks/20-full-res-model-all-angles-horizontal-cut-no-bbox-Copy1.ipynb#combining" data-toc-modified-id="combining-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>combining</a></span></li></ul></li></ul></div>

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import math
import random
import gzip
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Define loss functions

In [3]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [4]:
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [5]:
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

In [6]:
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [7]:
def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    return model

In [8]:
model = get_unet(input_shape=(640,1920,3), initial_filters=8)

In [9]:
model.load_weights('./weights/best_weights_fullres3.hdf5')

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 640, 1920, 3)  0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 640, 1920, 8)  224         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 640, 1920, 8)  32          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 640, 1920, 8)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

# Training

## Functions, generators and data

In [11]:
df_train = pd.read_csv('data/train_masks.csv')

In [12]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [13]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [14]:
ids_train_split = list(ids_train_split)
ids_valid_split = list(ids_valid_split)

In [15]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [16]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [17]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask


In [18]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:640, :, :])
                mask1 = np.copy(mask[0:640, :])
                mask1 = np.expand_dims(mask1, axis=2)
                
                img1  = np.concatenate([np.copy(img1[:,0:1,:]), img1, np.copy(img1[:,1917:1918,:])], axis = 1)
                mask1 = np.concatenate([np.copy(mask1[:,0:1,:]), mask1, np.copy(mask1[:,1917:1918,:])], axis = 1)
                
                img2  = np.copy(img[640:, :, :])
                mask2 = np.copy(mask[640:, :])
                mask2 = np.expand_dims(mask2, axis=2)
                
                img2  = np.concatenate([np.copy(img2[:,0:1,:]), img2, np.copy(img2[:,1917:1918,:])], axis = 1)
                mask2 = np.concatenate([np.copy(mask2[:,0:1,:]), mask2, np.copy(mask2[:,1917:1918,:])], axis = 1)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [50]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:640, :, :])
                mask1 = np.copy(mask[0:640, :])
                mask1 = np.expand_dims(mask1, axis=2)
                
                img1  = np.concatenate([np.copy(img1[:,0:1,:]), img1, np.copy(img1[:,1917:1918,:])], axis = 1)
                mask1 = np.concatenate([np.copy(mask1[:,0:1,:]), mask1, np.copy(mask1[:,1917:1918,:])], axis = 1)
                
                img2  = np.copy(img[640:, :, :])
                mask2 = np.copy(mask[640:, :])
                mask2 = np.expand_dims(mask2, axis=2)
                
                img2  = np.concatenate([np.copy(img2[:,0:1,:]), img2, np.copy(img2[:,1917:1918,:])], axis = 1)
                mask2 = np.concatenate([np.copy(mask2[:,0:1,:]), mask2, np.copy(mask2[:,1917:1918,:])], axis = 1)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

## Training

In [20]:
model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [52]:
val_batch_size = 16

In [32]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.61455859416594438, 0.8716211766297316]

In [34]:
model.load_weights('./weights/horiz_cut.hdf5')

In [35]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.012835991365765541, 0.99428087590017111]

In [42]:
train_batch_size = 4
max_epochs = 50

In [43]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_640.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
2661s - loss: 0.0072 - dice_coeff: 0.9963 - val_loss: 0.0073 - val_dice_coeff: 0.9963
Epoch 2/50


KeyboardInterrupt: 

In [49]:
model.load_weights('./weights/horiz_cut_640.hdf5')

In [53]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0072986759814435585, 0.99634190041330328]

# Only the top part

In [38]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [39]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                x_batch.append(img1)
                y_batch.append(mask1)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [40]:
train_batch_size = 6
val_batch_size   = 24

In [41]:
model.load_weights('weights/horiz_cut.hdf5')

In [42]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0059809478709734023, 0.99730878869302142]

In [43]:
model.compile(optimizer=RMSprop(lr=0.00001), loss=bce_dice_loss, metrics=[dice_coeff])

In [45]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_top.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=int(np.ceil(float(len(ids_train_split)) / float(train_batch_size))),
                    epochs=15,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=int(np.ceil(float(len(ids_valid_split)) / float(val_batch_size))))

Epoch 1/15
1585s - loss: 0.0078 - dice_coeff: 0.9967 - val_loss: 0.0058 - val_dice_coeff: 0.9973
Epoch 2/15
1583s - loss: 0.0061 - dice_coeff: 0.9971 - val_loss: 0.0057 - val_dice_coeff: 0.9973
Epoch 3/15
1582s - loss: 0.0060 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9973
Epoch 4/15
1584s - loss: 0.0059 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 5/15
1583s - loss: 0.0059 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 6/15

Epoch 00005: reducing learning rate to 9.999999747378752e-07.
1583s - loss: 0.0059 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 7/15
1583s - loss: 0.0058 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 8/15
1583s - loss: 0.0058 - dice_coeff: 0.9972 - val_loss: 0.0056 - val_dice_coeff: 0.9974
Epoch 00007: early stopping


# Only the bottom part

In [54]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img2  = np.copy(img[640:, :, :])
                mask2 = np.copy(mask[640:, :])
                mask2 = np.expand_dims(mask2, axis=2)
                
                img2  = np.concatenate([np.copy(img2[:,0:1,:]), img2, np.copy(img2[:,1917:1918,:])], axis = 1)
                mask2 = np.concatenate([np.copy(mask2[:,0:1,:]), mask2, np.copy(mask2[:,1917:1918,:])], axis = 1)
                
                                
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [55]:
val_batch_size   = 32

In [61]:
model.load_weights('weights/horiz_cut_640.hdf5')

In [62]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0084927444435904907, 0.99592590097828326]

In [63]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                
                img2  = np.copy(img[640:, :, :])
                mask2 = np.copy(mask[640:, :])
                mask2 = np.expand_dims(mask2, axis=2)
                
                img2  = np.concatenate([np.copy(img2[:,0:1,:]), img2, np.copy(img2[:,1917:1918,:])], axis = 1)
                mask2 = np.concatenate([np.copy(mask2[:,0:1,:]), mask2, np.copy(mask2[:,1917:1918,:])], axis = 1)
                
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [ ]:
train_batch_size = 8
max_epochs = 50

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/horiz_cut_bottom_640.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=int(np.ceil(float(len(ids_train_split)) / float(train_batch_size))),
                    epochs=10,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=int(np.ceil(float(len(ids_valid_split)) / float(val_batch_size))))

Epoch 1/10
1329s - loss: 0.0114 - dice_coeff: 0.9948 - val_loss: 0.0089 - val_dice_coeff: 0.9955
Epoch 2/10
1329s - loss: 0.0086 - dice_coeff: 0.9957 - val_loss: 0.0081 - val_dice_coeff: 0.9959
Epoch 3/10
1327s - loss: 0.0085 - dice_coeff: 0.9957 - val_loss: 0.0085 - val_dice_coeff: 0.9958
Epoch 4/10


# Is the specialised model much better than the common model

## top part

In [25]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                x_batch.append(img1)
                y_batch.append(mask1)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [26]:
model.load_weights('weights/horiz_cut.hdf5')

In [27]:
val_batch_size   = 24

In [28]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0059811309636854122, 0.99730860519502862]

In [29]:
model.load_weights('weights/horiz_cut_top.hdf5')

In [30]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.005567687237909711, 0.99738266952848154]

## bottom part

In [31]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                                
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [32]:
model.load_weights('weights/horiz_cut.hdf5')

In [33]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0080095959391832828, 0.99667244798308041]

In [34]:
model.load_weights('weights/horiz_cut_bottom2.hdf5')

In [35]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.0074096983024202773, 0.99673010523052252]

# Test set predictions

## top part

In [20]:
model.load_weights('weights/horiz_cut_top.hdf5')

In [21]:
from tqdm import tqdm

In [22]:
df_test = pd.read_csv('data/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

In [23]:
names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))

In [24]:
import bcolz

In [25]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [26]:
def save_test_predictions(ids_test, fname):
    val_batch_size = 24
    all_preds = []
    for start in range(0, len(ids_test), val_batch_size):
        x_batch = []
        end = min(start + val_batch_size, len(ids_test))
        ids_test_batch = ids_test[start:end]
        for id in ids_test_batch.values:
            img = cv2.imread('data/test/{}.jpg'.format(id))
            img1  = np.copy(img[0:768, :, :])
            img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
            x_batch.append(img1)
        x_batch = np.array(x_batch, np.float32) / 255
        preds = model.predict_on_batch(x_batch)
        preds = np.squeeze(preds, axis=3)
        all_preds.append(preds)
    all_preds = np.concatenate(all_preds, axis=0)
    all_preds = (all_preds*100).astype(np.uint8)
    save_array(fname, all_preds)

In [27]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    save_test_predictions(ids_test_batch, './horiz-cut-top-preds/batch-' + str(start))

100%|██████████| 401/401 [3:11:08<00:00, 22.82s/it]  


## Bottom part

In [28]:
model.load_weights('weights/horiz_cut_bottom2.hdf5')

In [29]:
from tqdm import tqdm

In [17]:
df_test = pd.read_csv('data/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

In [18]:
names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))

In [32]:
import bcolz

In [33]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [35]:
def save_test_predictions(ids_test, fname):
    val_batch_size = 24
    all_preds = []
    for start in range(0, len(ids_test), val_batch_size):
        x_batch = []
        end = min(start + val_batch_size, len(ids_test))
        ids_test_batch = ids_test[start:end]
        for id in ids_test_batch.values:
            img = cv2.imread('data/test/{}.jpg'.format(id))
            img2  = np.copy(img[512:1280, :, :])
            img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
            x_batch.append(img2)
        x_batch = np.array(x_batch, np.float32) / 255
        preds = model.predict_on_batch(x_batch)
        preds = np.squeeze(preds, axis=3)
        all_preds.append(preds)
    all_preds = np.concatenate(all_preds, axis=0)
    all_preds = (all_preds*100).astype(np.uint8)
    save_array(fname, all_preds)

In [36]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    save_test_predictions(ids_test_batch, './horiz-cut-bottom-preds/batch-' + str(start))

100%|██████████| 401/401 [3:10:49<00:00, 22.29s/it]  


## combining

In [10]:
from tqdm import tqdm

In [25]:
# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle


rles = []

In [12]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [38]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    
    top_part = load_array('./horiz-cut-top-preds/batch-'+str(start))
    bottom_part = load_array('./horiz-cut-bottom-preds/batch-'+str(start))
    
    top_part = top_part[:,:,2:]
    bottom_part = bottom_part[:,:,0:1918]
    
    part_1 = top_part[:,0:512,:]
    part_2_1 = top_part[:,512:,:]
    part_2_2 = bottom_part[:,0:256,:]
    part_3 = bottom_part[:,256:,:]
    part_2 = np.add(part_2_1, part_2_2)/2
    all_parts = np.concatenate((part_1, part_2, part_3), axis=1)
    all_parts = all_parts > 50
    
    for i in range(250):
        mask = all_parts[i,:,:]
        rle = run_length_encode(mask)
        rles.append(rle)

100%|█████████▉| 400/401 [40:10<00:05,  5.90s/it]

IndexError: index 64 is out of bounds for axis 0 with size 64

In [40]:
print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission13.csv.gz', index=False, compression='gzip')

Generating submission file...


In [19]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    
    top_part = load_array('./horiz-cut-top-preds/batch-'+str(start))
    bottom_part = load_array('./horiz-cut-bottom-preds/batch-'+str(start))
    
    top_part = top_part[:,:,2:]
    bottom_part = bottom_part[:,:,0:1918]
    
    part_1   = np.copy(top_part[:,0:512,:]) > 50
    part_2_1 = np.copy(top_part[:,512:,:]) > 50
    part_2_2 = np.copy(bottom_part[:,0:256,:]) > 50
    part_2   = np.add(part_2_1, part_2_2)
    part_3   = np.copy(bottom_part[:,256:,:]) > 50
    all_parts = np.concatenate((part_1, part_2, part_3), axis=1)
    
    for i in range(250):
        mask = all_parts[i,:,:]
        rle = run_length_encode(mask)
        rles.append(rle)

100%|█████████▉| 400/401 [21:07<00:03,  3.19s/it]

IndexError: index 64 is out of bounds for axis 0 with size 64

In [20]:
print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission14.csv.gz', index=False, compression='gzip')

Generating submission file...


In [26]:
for start in tqdm(range(0, len(ids_test), 250)):
    end = min(start + 250, len(ids_test))
    ids_test_batch = ids_test[start:end]
    
    top_part = load_array('./horiz-cut-top-preds/batch-'+str(start))
    bottom_part = load_array('./horiz-cut-bottom-preds/batch-'+str(start))
    
    top_part = top_part[:,:,2:]
    bottom_part = bottom_part[:,:,0:1918]
    
    part_1   = np.copy(top_part[:,0:512,:]) > 50
    part_2_1 = np.copy(top_part[:,512:,:]) > 50
    part_2_2 = np.copy(bottom_part[:,0:256,:]) > 50
    part_2   = np.logical_and(part_2_1, part_2_2)
    part_3   = np.copy(bottom_part[:,256:,:]) > 50
    all_parts = np.concatenate((part_1, part_2, part_3), axis=1)
    
    for i in range(250):
        mask = all_parts[i,:,:]
        rle = run_length_encode(mask)
        rles.append(rle)


100%|█████████▉| 400/401 [20:14<00:03,  3.05s/it]

IndexError: index 64 is out of bounds for axis 0 with size 64

In [27]:
print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission15.csv.gz', index=False, compression='gzip')

Generating submission file...
